In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from pymongo import MongoClient

In [14]:
client = MongoClient('localhost', 27017)
db = client['BD2']
mails = db.mails

chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://mail.yandex.ru/')

button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'button2_theme_mail-white')))
button.click()

assert "Авторизация" in driver.title
elem = driver.find_element_by_id('passp-field-login')
elem.send_keys('') #Здесь должен быть логин
elem.send_keys(Keys.RETURN)
time.sleep(2)
elem = driver.find_element_by_id('passp-field-passwd')
elem.send_keys('')
elem.send_keys(Keys.RETURN)
time.sleep(3)


# # WebDriverWait(driver,10).until(
# #             EC.element_to_be_clickable((By.CLASS_NAME, 'js-href b-datalist__item__link')))
assert "Входящие" in driver.title
i = 0
while True:
    try:
        dict_of_data = {}
        froms = driver.find_elements_by_class_name('ns-view-messages-item-wrap')
        froms[i].click()
        time.sleep(2)
        sender = driver.find_element_by_class_name('mail-ui-HoverLink-Content').get_attribute('title')
        date = driver.find_element_by_class_name('ns-view-message-head-date').text
        subject = driver.find_element_by_class_name('mail-Message-Toolbar-Subject').text
        driver.back()
        time.sleep(2)
        dict_of_data['sender'] = sender
        dict_of_data['date'] = date
        dict_of_data['subject'] = subject
        mails.insert_one(dict_of_data)
        i += 1
    except Exception as e:
        print(e)
        break

list index out of range
